In [1]:
import pyspark

In [2]:
pyspark.__file__

'/home/kiritsugulyn/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/28 12:12:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.version

'3.3.2'

In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-02-28 12:13:14--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.40, 3.164.82.112, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  13.5MB/s    in 5.8s    

2025-02-28 12:13:21 (10.6 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [7]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

In [8]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [9]:
df.repartition(4).write.parquet("yellow_tripdata_2024-10_partitioned.parquet")

In [14]:
!cd yellow_tripdata_2024-10_partitioned.parquet && du -sch *.parquet

25M	part-00000-dd20e09b-b759-4e6b-81e4-b915830bd54f-c000.snappy.parquet
25M	part-00001-dd20e09b-b759-4e6b-81e4-b915830bd54f-c000.snappy.parquet
25M	part-00002-dd20e09b-b759-4e6b-81e4-b915830bd54f-c000.snappy.parquet
25M	part-00003-dd20e09b-b759-4e6b-81e4-b915830bd54f-c000.snappy.parquet
97M	total


In [55]:
from pyspark.sql import functions as F
df.filter(F.to_date(F.col("tpep_pickup_datetime")) == F.to_date(F.lit("2024-10-15"))).count()

128893

In [28]:
df.select(F.max((F.col("tpep_dropoff_datetime") - F.col("tpep_pickup_datetime")).cast('long')/3600)).show()

+----------------------------------------------------------------------------+
|max((CAST((tpep_dropoff_datetime - tpep_pickup_datetime) AS BIGINT) / 3600))|
+----------------------------------------------------------------------------+
|                                                          162.61777777777777|
+----------------------------------------------------------------------------+



In [40]:
df.groupBy('PULocationID').count().orderBy(F.col("count").asc()).limit(1).show()

+------------+-----+
|PULocationID|count|
+------------+-----+
|         105|    1|
+------------+-----+



In [30]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-02-28 13:02:24--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.112, 3.164.82.40, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-02-28 13:02:25 (309 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [45]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
])

In [46]:
df_zone = spark.read.option('header','true').schema(schema).csv("taxi_zone_lookup.csv")

In [47]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [51]:
df_zone.filter(F.col('LocationID') == 105).select(F.col('Zone')).show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
+--------------------+

